In [2]:
from ultralytics import YOLO
import cv2
import math
import torch
import numpy as np

In [3]:
# GENERIC_WEIGHTS = "D:/RajHack/Rakshak/video-server/models/weights/yolo_v8/yolov8n.pt"
FIRE_WEIGHTS = "firedetection.pt"
# WEAPONS_WEIGHTS = "D:/RajHack/Rakshak/video-server/models/weights/weapon_detection/weapons.pt"

In [4]:
classes = {
    "generic": ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ],
    "fire" : ["fire","smoke"],
    "weapons" : ['Grenade', 'Knife', 'Missile', 'Pistol', 'Rifle']
}

In [5]:
models = {}
models["generic"] = YOLO("yolov8n.pt")
models["fire"] = YOLO(FIRE_WEIGHTS)
# models["weapons"] = YOLO(WEAPONS_WEIGHTS)

In [6]:
import time

In [7]:
# inmatesthresholdtime=5
# def process_model(model, img, classNames, model_name, numinmates, savetodir="anomalyframes"):
#     global inmatesthresholdtime
#     results = model(img, stream=True)
#     inmates = 0
#     for r in results:
#         boxes = r.boxes
#         anomalies = []
#         for box in boxes:
#             # bounding box
#             cls = int(box.cls[0])
#             if cls >= len(classNames):
#                 print("Error: Class index out of range")
#                 continue
#             accepted_list = ["fire","smoke", 'Grenade', 'Knife', 'Pistol', 'Rifle', 'scissors', 'cell phone', 'knife']
#             # if classNames[cls] not in accepted_list:
#             #     print("Class not in accepted list")
#             #     continue
          
#             x1, y1, x2, y2 = map(int, box.xyxy[0])
                
#             print("x1,y1,x2,y2",x1,y1,x2,y2)

#             cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
#             if model_name == "generic" and classNames[cls] == "person":
#                 inmates += 1

#             if inmates != numinmates and  inmatesthresholdtime < 0:
#                 anomalies.append(f"Inmates are not in the right number, current number of inmates is {inmates}, storing the frame in the database as {time.time()}.jpg")
#                 inmatesthresholdtime = 5
#                 cv2.imwrite(f"{savetodir}/{time.time()}.jpg", img)
#             elif inmates != numinmates and inmatesthresholdtime > 0 and classNames[cls] == "person":
#                 inmatesthresholdtime -= 1

#             confidence = math.ceil((box.conf[0]*100))/100
#             print("Confidence --->",confidence)
#             print("Class name -->", classNames[cls])

#             org = [x1, y1]
#             font = cv2.FONT_HERSHEY_SIMPLEX
#             fontScale = 1
#             color = (255, 0, 0)
#             thickness = 2

#             cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)
#         with open("anomalies.txt", "a") as f:
#             f.write("\n".join(anomalies))


# def start_webcam(models, classes):
#     cap = cv2.VideoCapture(0)
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             print("Error: Unable to capture frame")
#             break
#         process_model(models["generic"], frame, classes["generic"], "generic",2)
#         process_model(models["fire"], frame, classes["fire"], "fire",2)
#         # process_model(models["weapons"], frame, classes["weapons"], "weapons", 1)
#         cv2.imshow("frame", frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):

#             break
#     cap.release()
#     cv2.destroyAllWindows()


In [8]:
# start_webcam(models, classes)

In [46]:
import pandas as pd

def create_csv():
  try:
    df = pd.read_csv('anomalies.csv')
  except FileNotFoundError:
    df = pd.DataFrame(columns=['Time', 'Anomaly message','Type of anomaly' ,'Number of Inmates in Jail', 'ImagePath'])
    df.to_csv('anomalies.csv', index=False)
  return df


def append_to_csv(df, timestamp, anomaly_text, num_inmates, image_path, typeofanomaly=""):
  new_data = {'Time': timestamp, 'Anomaly message': anomaly_text,'Type of anomaly':typeofanomaly, 'Number of Inmates in Jail': num_inmates, 'ImagePath': image_path}
  new_data_df = pd.DataFrame.from_dict(new_data, orient='index').T
  df = pd.concat([df, new_data_df], ignore_index=True)
  df.to_csv('anomalies.csv', index=False)

inmatesthresholdtime = 15
anomaly_count = {} 


def process_model(model, img, classNames, model_name, numinmates, savetodir="anomalyframes"):
  global inmatesthresholdtime, anomaly_count
  results = model(img, stream=True)
  inmates = 0
  anomalies = []
  for r in results:
    boxes = r.boxes
    for box in boxes:
      # bounding box
      cls = int(box.cls[0])
      if cls >= len(classNames):
        print("Error: Class index out of range")
        continue
      accepted_list = ['Grenade', 'Knife', 'Pistol', 'Rifle', 'scissors', 'cell phone', 'knife']

      x1, y1, x2, y2 = map(int, box.xyxy[0])
      
      cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
      if model_name == "generic" and classNames[cls] == "person":
        inmates += 1

      # Track anomaly count for current model and class
      if classNames[cls] in accepted_list:
        anomaly_key = (model_name, classNames[cls])
        anomaly_count[anomaly_key] = anomaly_count.get(anomaly_key, 0) + 1

      confidence = math.ceil((box.conf[0]*100))/100
      print("Confidence --->",confidence)
      print("Class name -->", classNames[cls])

      org = [x1, y1]
      font = cv2.FONT_HERSHEY_SIMPLEX
      fontScale = 1
      color = (255, 0, 0)
      thickness = 2

      cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)

  if inmates != numinmates and model_name == "generic" and inmatesthresholdtime <= 0:
        timestamp = time.strftime('%Y-%m-%d %H:%M:%S') 
        anomaly_text = f"Inmates are not in the right number, current number of inmates is {inmates}, expected number was {numinmates} , storing the frame in the database as {savetodir}/{timestamp}.jpg"
        print(anomaly_text)
        anomaly_count = {key: 0 for key in anomaly_count.keys()}
        anomalies.append(anomaly_text)
        inmatesthresholdtime = 15
        df = create_csv()
        append_to_csv(df, timestamp, anomaly_text, inmates, f"{savetodir}/{timestamp}.jpg", "Inmate count anomaly")
        cv2.imwrite(f"{savetodir}/{timestamp}.jpg", img)
  elif inmates != numinmates and inmatesthresholdtime > 0:
        inmatesthresholdtime -= 1
  with open("anomalies.txt", "a") as f:
     f.write("\n".join(anomalies) + "\n")

     
def start_webcam(models, classes):
  cap = cv2.VideoCapture(0)
  while True:
    ret, frame = cap.read()
    if not ret:
      print("Error: Unable to capture frame")
      break
    for model_name, model in models.items():
      process_model(model, frame.copy(), classes[model_name], model_name, 1)  # Pass a copy of frame to avoid modification across models
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break

  cap.release()
  cv2.destroyAllWindows()


In [48]:
start_webcam(models=models, classes=classes)


0: 480x640 1 person, 172.4ms
Confidence ---> 0.85
Class name --> person
Speed: 0.0ms preprocess, 172.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 608x800 1 default, 1 smoke, 482.6ms
Confidence ---> 0.35
Class name --> smoke
Error: Class index out of range
Speed: 8.1ms preprocess, 482.6ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)

0: 480x640 1 person, 133.0ms
Confidence ---> 0.85
Class name --> person
Speed: 0.0ms preprocess, 133.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 608x800 (no detections), 394.6ms
Speed: 0.0ms preprocess, 394.6ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)

0: 480x640 1 person, 118.0ms
Confidence ---> 0.88
Class name --> person
Speed: 3.0ms preprocess, 118.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 608x800 1 default, 373.6ms
Confidence ---> 0.59
Class name --> smoke
Speed: 8.0ms preprocess, 373.6ms inference, 0.0ms postprocess per i

In [51]:
import lida
from lida import llm, Manager, TextGenerationConfig
import pandas as pd
import os

In [67]:
os.environ["COHERE_API_KEY"] = "hk305qjjFaCIXME1dGS0OIIHDth4NFnTtuzpPCOg"

In [80]:
# Load your data (replace with your data source)
data = pd.read_csv("anomalies.csv")

lida = Manager(text_gen = llm("cohere")) 
textgen_config = TextGenerationConfig(n=3, temperature=0.9, use_cache=True)

summary = lida.summarize(data=data, summary_method="default", textgen_config=textgen_config)  
goals = lida.goals(summary, n=2, textgen_config=textgen_config)

for goal in goals:
    display(goal)# Save as image


### Goal 0
---
**Question:** What is the distribution of the Time of anomaly detection?

**Visualization:** `Histogram of Time`

**Rationale:** The histogram of the time can show the distribution of the anomaly detections and highlight any patterns or clusters in the anomalies



### Goal 1
---
**Question:** What is the correlation between the image path and the type of anomaly?

**Visualization:** `Scatter plot of ImagePath vs Type of Anomaly`

**Rationale:** There might be types of anomalies that need specific data to be identified. This can help us understand if certain types of anomalies can be predicted based on the data that is captured.


In [81]:

# textgen_config = TextGenerationConfig(n=1, temperature=0.2)
# persona = "I am a security guard at a prison. I want to be able to detect anomalies in the prison environment."
# personal_goals = lida.goals(summary, n=2, persona=persona, textgen_config=textgen_config)
# for goal in personal_goals:
#     display(goal)

i = 0
library = "seaborn"

charts = lida.visualize(summary=summary, goal=goal, textgen_config=textgen_config, library=library)  

# Display or save the chart
charts[0].show()  # Display in notebook
charts[0].savefig("chart.png")  # Save as image

 ```python
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def plot(data: pd.DataFrame):
    anomaly_message = data['Anomaly message'].str.split(',', expand=True)
    anomaly_message = anomaly_message[1].str.strip()
    data['Type of Anomaly'] = anomaly_message

    anomaly_image = data['ImagePath'].str.contains('anomaly')
    data['Type of Anomaly'] = data['Type of Anomaly'].astype('category')
    data = data[anomaly_image]

    plt.title('What is the correlation between the image path and the type of anomaly?', wrap=True)
    sns.scatterplot(data=data, x='ImagePath', y='Type of Anomaly', hue='Type of Anomaly')
    plt.legend(title='Type of Anomaly')
    return plt

chart = plot(data)
```
This code uses the `seaborn` library to create a scatter plot of `ImagePath` vs. `Type of Anomaly` and adds a legend to the plot. It also ensures that the plot includes only valid data fields. 
****
 module 'numpy' has no attribute 'matrix'
 ```python
import seaborn as sns


IndexError: list index out of range